# Optical Inspection using TAO Siamese Network

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## What is a Siamese Network?

A Siamese Network is a class of neural network architectures that contain two or more identical subnetworks.
The training algorithm works by updating the parameters across all the sub-networks in tandem.
It is used to find the similarity between the inputs by computing the Euclidean distance between the feature vectors.
In this specific use case, the inputs are a "golden or reference" image and the image of the PCB component under inspection.

### Sample inference from the optical inspection siamese model

| **Pass** | **Fail** |
| :--: | :--: |
|<img align="center" title="no defects" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/optical_inspection/pass.png?raw=true" width="400" > no defects | <img align="center" title="missing component" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/optical_inspection/defect.png?raw=true" width="400" >  missing component |


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train a Siamese Network for Optical Inspection on PCB Dataset.
* Explore options to combine images of multiple LED light sources within Siamese Network 
* Evaluate the trained model & results.
* Run Inference on the trained model.
* Export the trained model to a .onnx file (encrypted ONNX model) for deployment to DeepStream or TensorRT.

At the end of this notebook, you will have generated a trained and optimized `opticalinspection` model, 
which you may deploy with this [end-to-end sample](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps) with Triton.

Siamese Network TAO Features:

* Combine multiple LED intensities, camera angles or different sensory inputs for image-pair comparison within Siamese Network
    * Multiple Inputs (1....N)
* Mode of concat
    * Linear concat (1 X N)
    * Grid concat (N X M)
* Backbone networks supported for Siamese
    * Custom
* Backbone with pre-trained weights
    * Yes
    * No


## Table of Contents

This notebook shows an example usecase of Siamese Network using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset](#head-2)
3. [Download the pretrained model from NGC](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Inferences](#head-7)
8. [Deploy](#head-8)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=FIXME

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "optical_inspection")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "optical_inspection", "results")
os.environ["HOST_MODEL_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "optical_inspection", "model")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=/path/to/local/tao-experiments/optical_inspection
# The sample spec files are present in the same path as the downloaded samples.
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR
! mkdir -p $HOST_MODEL_DIR

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       # Mapping the data directory
       {
           "source": os.environ["LOCAL_PROJECT_DIR"],
           "destination": "/workspace/tao-experiments"
       },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_MODEL_DIR"],
           "destination": "/model"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

## 2. Prepare a sample dataset <a class="anchor" id="head-2"></a>

TAO optical inspection trainer uses a custom dataset format. The sections below walk through this format, specifically how the dataset should be structured and the files required.

### Optical Inspection data format

Optical Inspection expects directories of images and csv files in the dataset root directory. The image directory consists of golden images (non-defective reference images) and test images to be
compared with golden samples for PCB defect classification.  


    |--dataset_root:
       |--images
            |--input:
               |--C1.jpg
               |--C2.jpg
            |--golden:
               |--C1.jpg
               |--C2.jpg
            |--input1:
               |--C1.jpg
               |--C3.jpg
            |--golden1:
               |--C1.jpg
               |--C3.jpg
       |--labels
           |-- train.csv
           |-- validation.csv

Here's a description of the structure:

* The images directory contains the following:
    * input: Directory containing input images to be compared with golden images
    * golden: Directory containing golden reference images
* The labels directory contains the csv files for pair-wise images input to the SiameseOI model with corresponding class labels. Details of this file are included in the `Label Files` section below. 

#### Label Files

A SiameseOI label file is a simple csv file containing the following fields.

| **input_path** | **golden_path** | **label** | **object_name** |
| :-- | :-- | :--- | :--- |
| `/path/to/input/image/directory` | `/path/to/golden/image/directory` | The class to which the object belongs | `/component/name` |

Here's a description of the structure:

* ``input_path``: The path to the directory containing input compare image
* ``golden_path``: The path to the directory containing corresponding golden reference image
* ``label``: The labels for the pair-wise images (Use `PASS` for non-defective components, and any other specific defect type label for defective components).
* ``object_name``: The name of the component to be compared. The object name is the same for input and golden images and represents the image name without the file extension.
    * For each :code:`object_name`, TAO supports combining multiple LED intensities, camera angles or different sensory inputs for each of the input and golden images to be
      compared within the SiameseOI model. For more details, refer to the :ref:`Input Mapping<input_map_siamese>` section below.

Here is a sample label file corresponding to the sample directory structure as describe in the :ref:`Optical Inspection Format <optical_inspection_format>` section.

| **input_path** | **golden_path** | **label** | **object_name** |
| :-- | :-- | :--- | :--- |
| /dataset_root/images/input/  | /dataset_root/images/golden/  | PASS      | C1  |
| /dataset_root/images/input/  | /dataset_root/images/golden/  | PASS      | C2  |
| /dataset_root/images/input1/ | /dataset_root/images/golden1/ | MISSING   | C1  |
| /dataset_root/images/input1/ | /dataset_root/images/golden1/ | PASS      | C3  |

``Note``: In the label file, ensure that non-defective samples are consistently labeled as PASS, 
while defective samples can be assigned any specific defect type label. 
The model is designed to treat all defects collectively and train for binary defect classification.

#### Input Mapping

TAO supports combining several lighting conditions (1...N) for each image-golden pair to be combined for comparison within Siamese Network. The following modes are supported: 

* Linear concat (1 x N)
* Grid concat (M x N)

Each `object_name` is appended with name of the lighting condition as specified in the experiment spec under `input_map`. Here is an example of dataset experiment spec changes for combining 4 input lighting conditions as a 1x4 linear grid:

```yaml
    dataset: 
        num_input: 4
        concat_type: linear
        input_map:
            LowAngleLight: 0
            SolderLight: 1
            UniformLight: 2
            WhiteLight: 3
```

The dataset also supports single lighting condition per input-golden image pair as input to the Siamese Network. Here is an example of dataset experiment spec changes for single lighting condition where `object_name` represents the image name:

```yaml
    dataset: 
        num_inputs: 1
        input_map: null
```


In [ ]:
# Verify
!ls -l $HOST_DATA_DIR/

# 3. Download the pretrained model from NGC <a class="anchor" id="head-3"></a>

Download the pretrained model from NGC. We will use NGC CLI to get the data and model. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $HOST_RESULTS_DIR/ngccli

# # Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/optical_inspection:*

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
!ngc registry model download-version "nvidia/tao/optical_inspection:trainable_v1.0" --dest $HOST_RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/optical_inspection_vtrainable_v1.0

## 4. Provide training specification <a class="anchor" id="head-4"></a>

We provide specification files to configure the training parameters including:

* model:
  * model_type: Siamese_3
  * model_backbone: custom
  * embedding_vectors: 5
  * margin: 2.0
* dataset:
  * train_dataset:
    * csv_path: /data/dataset_convert/train_combined.csv
    * images_dir: /data/images/
  * validation_dataset:
    * csv_path: /data/dataset_convert/valid_combined.csv
    * images_dir: /data/images/
  * test_dataset:
    * csv_path: /data/dataset_convert/valid_combined.csv
    * images_dir: /data/images/
  * infer_dataset:
    * csv_path: /data/dataset_convert/valid_combined.csv
    * images_dir: /data/images/
  * image_ext: .jpg
  * batch_size: 4
  * workers: 1
  * fpratio_sampling: 0.1
  * num_input: 4
  * input_map:
    * LowAngleLight: 0
    * SolderLight: 1
    * UniformLight: 2
    * WhiteLight: 3
  * concat_type: linear
  * grid_map:
    * x: 2
    * y: 2
  * output_shape:
    * 128
    * 128
  * augmentation_config:
    * rgb_input_mean: [0.485, 0.456, 0.406]
    * rgb_input_std: [0.229, 0.224, 0.225]
* train:
  * optim:
    * type: Adam
    * lr: 0.0005
  * loss: contrastive
  * epochs: 5
  * checkpoint_interval: 5
  * results_dir: results/train
  * tensorboard:
    * enabled: True
 
Please refer to the TAO documentation about Optical Inspection to get all the parameters that are configurable.

In [ ]:
!cat $HOST_SPECS_DIR/experiment.yaml

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models.
* WARNING: Training will take several hours or one day to complete.

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.
# The data is saved here
%env DATA_DIR = /data
%env MODEL_DIR = /model
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

### 5.1 Train Siamese model

We will train a Siamese model from scratch.

In [ ]:
print("Train model")
!tao model optical_inspection train \
                  -e $SPECS_DIR/experiment.yaml \
                  train.pretrained_model_path=$RESULTS_DIR/pretrained/optical_inspection_vtrainable_v1.0/oi_model.pth

In [ ]:
## Training command for multi-gpu training. We can define the number of gpus and specify which GPU's are to be used by setting the `train.gpu_ids` parameter.
## The following command will trigger multi-gpu training on gpu 0 and gpu 1.
# !tao model optical_inspection train \
#                   -e $SPECS_DIR/experiment.yaml \
#                   train.gpu_ids=[0,1]

In [ ]:
print('Model checkpoints:')
!ls -ltrh $HOST_RESULTS_DIR/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
%env NUM_EPOCH=14

In [ ]:
# Get the name of the checkpoint corresponding to your set epoch
tmp=!ls $HOST_RESULTS_DIR/train/*.pth | grep epoch=0$NUM_EPOCH
%env CHECKPOINT={tmp[0]}

print('Rename a trained model: ')
print('---------------------')
!cp $CHECKPOINT $HOST_RESULTS_DIR/train/oi_model.pth
!ls -ltrh $HOST_RESULTS_DIR/train/oi_model.pth

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>
Evaluate trained model.

For Siamese model evaluation, we use the following metrics:

* Defect Accuracy: Defect detection accurary.
* False Positive Rate (FPR): False Alarm Rate for for non-defective samples.

In [ ]:
!tao model optical_inspection evaluate \
                   -e $SPECS_DIR/experiment.yaml

## 7. Inferences <a class="anchor" id="head-7"></a>
In this section, we run the optical inspection inference tool to generate inferences with the trained models and save the results under `$RESULTS_DIR`. 

In [ ]:
!tao model optical_inspection inference \
                   -e $SPECS_DIR/experiment.yaml

## 8. Deploy <a class="anchor" id="head-8"></a>
Export the model to encrypted ONNX model.

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
!tao model optical_inspection export \
                    -e $SPECS_DIR/experiment.yaml

In [ ]:
# # Uncomment this cell to export an onnx file with dynamic batching enabled for
# # integration with trtexec and deepstream.
# # Export the model with export.batch_size=-1 for dynamic batching.
# ! tao model optical_inspection export \
#                     -e $SPECS_DIR/experiment.yaml \
#                     export.batch_size=-1 \
#                     export.results_dir=$RESULTS_DIR/export_dynamic

# # Profiling the exported model via trtexec.
# ! tao deploy trtexec --onnx=$RESULTS_DIR/export_dynamic/oi_model.onnx \
#                      --minShapes=input_1:1x3x512x128,input_2:1x3x512x128 \
#                      --optShapes=input_1:8x3x512x128,input_2:8x3x512x128 \
#                      --maxShapes=input_1:16x3x512x128,input_2:16x3x512x128 \
#                      --fp16 \
#                      --saveEngine=$RESULTS_DIR/export_dynamic/oi_model_fp16.engine

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

In [ ]:
!tao deploy optical_inspection gen_trt_engine \
            -e $SPECS_DIR/experiment.yaml

In [ ]:
!tao deploy optical_inspection inference \
            -e $SPECS_DIR/experiment.yaml

You may continue by deploying the exported model to [TensorRT](https://developer.nvidia.com/tensorrt) for optimized inference.